<a href="https://www.kaggle.com/code/viccreate/notebook5beb0a3df9?scriptVersionId=224688516" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the datasets
df_playstore = pd.read_csv(r"C:\Users\Agbason\Desktop\HNG Internship\Task4\Google_Playstore.csv")

df_reviews = pd.read_csv(r"C:\Users\Agbason\Desktop\HNG Internship\Task4\reviews.csv")

In [ ]:
# Display basic info about the datasets

df_playstore.info(), df_reviews.info()

In [ ]:
# To display all columns
pd.set_option('display.max_columns', None)
df_playstore.head(5)

In [ ]:
pd.set_option('display.max_columns', None)
df_reviews.head(5)

Google Play Store Dataset (Apps Data)

Issues Identified:

Rating, Rating Count, Installs, Minimum Installs, and Currency contain missing values.

Installs is stored as an object (string) with values like "1+" and "5,000+", which need cleaning.

Date columns (Released, Last Updated, Scraped Time) should be converted to datetime format.

Some duplicate or unnecessary columns such as Scraped Time may not be useful for analysis.


Reviews Dataset

Issues Identified:

reviewCreatedVersion has missing values.

replyContent and repliedAt are mostly empty and might not be useful.

at (timestamp of review) should be converted to datetime format.

In [ ]:
df_playstore.duplicated().sum()

In [ ]:
df_reviews.duplicated().sum()

In [ ]:
df_playstore.isna().sum()

In [ ]:
df_reviews.isna().sum()

In [ ]:
df_playstore["Currency"].mode()

In [ ]:
# Cleaning Play Store dataset

# Convert "Installs" to numeric (remove "+" and ",")
df_playstore["Installs"] = df_playstore["Installs"].str.replace("+", "", regex=False).str.replace(",", "", regex=False).astype(float)

# Convert date columns to datetime format
df_playstore["Released"] = pd.to_datetime(df_playstore["Released"], errors='coerce')
df_playstore["Last Updated"] = pd.to_datetime(df_playstore["Last Updated"], errors='coerce')

# Fill missing numerical values with median
df_playstore["Rating"] = df_playstore["Rating"].fillna(df_playstore["Rating"].median())
df_playstore["Rating Count"] = df_playstore["Rating Count"].fillna(df_playstore["Rating Count"].median())

# Fill missing categorical values with mode
df_playstore["Currency"] = df_playstore["Currency"].fillna(df_playstore["Currency"].mode()[0])

# Drop irrelevant columns
df_playstore = df_playstore.drop(columns=["Scraped Time", "Developer Website"])

In [ ]:
# Cleaning Reviews dataset

# Convert "at" column to datetime format
df_reviews["at"] = pd.to_datetime(df_reviews["at"], errors='coerce')

# Drop unnecessary columns
df_reviews = df_reviews.drop(columns=["userImage", "replyContent", "repliedAt", "sortOrder"])

# Fill missing values in "reviewCreatedVersion" with "Unknown"
df_reviews["reviewCreatedVersion"] = df_reviews["reviewCreatedVersion"].fillna("Unknown")

# Standardizing column names for merging
df_playstore.rename(columns={"App Id": "appId"}, inplace=True)

In [ ]:
df_playstore.shape

In [ ]:
df_reviews.shape

In [ ]:
# Merge the datasets based on `appId`
df_merged = df_reviews.merge(df_playstore, on="appId", how="inner")

In [ ]:
# Display cleaned and merged data
df_merged.head(5)

In [ ]:
df_merged.shape

In [ ]:
# Text preprocessing and Sentiment Analysis
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

# Download VADER lexicon (if not already downloaded)
nltk.download('vader_lexicon')

# Initialize sentiment intensity analyzer
sia = SentimentIntensityAnalyzer()

# Function to classify sentiment
def get_sentiment(text):
    if not isinstance(text, str):
        return "Neutral"  # Handle NaN or non-string inputs

    score = sia.polarity_scores(text)["compound"]
    
    if score > 0:
        return "Positive"
    elif score < 0:
        return "Negative"
    else:
        return "Neutral"


# Apply sentiment analysis
df_merged["Sentiment"] = df_merged["content"].apply(get_sentiment)

In [ ]:
# Sentiment distribution
sentiment_counts = df_merged["Sentiment"].value_counts()
sentiment_counts

In [ ]:
# Display sentiment distribution
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
sentiment_counts.plot(kind="bar", title="Sentiment Distribution of User Reviews")
plt.xlabel("Sentiment")
plt.ylabel("Count")
plt.show()

Defining 10 Unique Business-Critical KPIs that are crucial for analyzing app performance, user engagement, and business impact.

App Performance & User Engagement KPIs:

1.Average Rating - The mean rating across all user reviews.

2.Total Number of Reviews - Count of all reviews per app.

3.Review Sentiment Distribution - Percentage breakdown of positive, neutral, and negative reviews per app.

4.Review Engagement Score - Weighted score based on thumbs-up counts on reviews.

5.App Install Growth Rate - Monthly growth rate in the number of installs.

6.Rating Volatility - Standard deviation of user ratings (indicates fluctuations in user satisfaction).

7.User Retention Score - Estimated based on long-term positive reviews and recurring user feedback.

8.In-App Purchase Influence - Correlation between price, in-app purchases, and sentiment analysis.

Predictive Analytics-Oriented KPIs:

9.Predicted Rating Trend - Forecasting future ratings using regression models.

10.Predictive Review Volume - Estimating future review count trends based on historical data.

In [ ]:
# KPI 1: Average Rating per App
avg_rating = df_merged.groupby("App Name")["score"].mean().reset_index()
avg_rating.columns = ["App Name", "Average Rating"]
avg_rating.head()

In [ ]:
# KPI 2: Total Number of Reviews per App
total_reviews = df_merged.groupby("App Name")["reviewId"].count().reset_index()
total_reviews.columns = ["App Name", "Total Reviews"]
total_reviews.head()

In [ ]:
# KPI 3: Review Sentiment Distribution per App
sentiment_distribution = df_merged.groupby(["App Name", "Sentiment"])["reviewId"].count().unstack(fill_value=0)
sentiment_distribution["Positive %"] = (sentiment_distribution.get("Positive", 0) / sentiment_distribution.sum(axis=1)) * 100
sentiment_distribution["Neutral %"] = (sentiment_distribution.get("Neutral", 0) / sentiment_distribution.sum(axis=1)) * 100
sentiment_distribution["Negative %"] = (sentiment_distribution.get("Negative", 0) / sentiment_distribution.sum(axis=1)) * 100
sentiment_distribution = sentiment_distribution.reset_index()

sentiment_distribution.head()

In [ ]:
# KPI 4: Review Engagement Score (Weighted score based on thumbs up count)
review_engagement = df_merged.groupby("App Name")["thumbsUpCount"].sum().reset_index()
review_engagement.columns = ["App Name", "Total Thumbs Up"]

review_engagement.head()

In [ ]:
# KPI 5: App Install Growth Rate
df_merged["Last Updated"] = pd.to_datetime(df_merged["Last Updated"])
install_growth = df_merged.groupby("App Name").agg({"Installs": "max", "Minimum Installs": "min"})
install_growth["Growth Rate %"] = ((install_growth["Installs"] - install_growth["Minimum Installs"]) / install_growth["Minimum Installs"]) * 100
install_growth.reset_index(inplace=True)
install_growth.head()

In [ ]:
# KPI 6: Rating Volatility (Standard deviation of ratings per app)
rating_volatility = df_merged.groupby("App Name")["score"].std().reset_index()
rating_volatility.columns = ["App Name", "Rating Volatility"]

rating_volatility.head()

In [ ]:
# KPI 7: User Retention Score (Based on % of positive reviews)
user_retention = sentiment_distribution[["App Name", "Positive %"]].copy()
user_retention.columns = ["App Name", "User Retention Score (%)"]

user_retention.head()

In [ ]:
# KPI 8: Influence of In-App Purchases (Average sentiment score for apps with purchases vs. free apps)
in_app_purchase_impact = df_merged.groupby("In App Purchases")["score"].mean().reset_index()
in_app_purchase_impact.columns = ["In App Purchases", "Average Rating"]

in_app_purchase_impact.head()

In [ ]:
# Merge KPIs into a single dataset
kpi_data = avg_rating.merge(total_reviews, on="App Name") \
    .merge(sentiment_distribution, on="App Name") \
    .merge(review_engagement, on="App Name") \
    .merge(install_growth[["App Name", "Growth Rate %"]], on="App Name") \
    .merge(rating_volatility, on="App Name") \
    .merge(user_retention, on="App Name")

### Display KPI insights

In [ ]:

# Function to visualize KPI data
def visualize_kpi_data(kpi_data):
    fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(20, 18))
    fig.suptitle("App Performance KPIs", fontsize=18, fontweight="bold")

    # Bar Chart: Average Rating
    kpi_data.plot(kind="bar", x="App Name", y="Average Rating", ax=axes[0, 0], color="skyblue", edgecolor="black")
    axes[0, 0].set_title("Average Rating per App")
    axes[0, 0].set_ylabel("Rating (0-5)")
    axes[0, 0].tick_params(axis="x", rotation=90,labelsize=6)

    # Bar Chart: Total Reviews
    kpi_data.plot(kind="bar", x="App Name", y="Total Reviews", ax=axes[0, 1], color="orange", edgecolor="black")
    axes[0, 1].set_title("Total Reviews per App")
    axes[0, 1].set_ylabel("Review Count")
    axes[0, 1].tick_params(axis="x", rotation=90,labelsize=6)

    # Pie Chart: Sentiment Distribution (Aggregated)
    sentiment_distribution = kpi_data[["Positive", "Neutral", "Negative"]].sum()
    axes[1, 0].pie(
        sentiment_distribution, labels=["Positive", "Neutral", "Negative"], 
        autopct="%1.1f%%", startangle=140, colors=["green", "gray", "red"]
    )
    axes[1, 0].set_title("Overall Review Sentiment Distribution")

    # Bar Chart: Review Engagement Score
    kpi_data.plot(kind="bar", x="App Name", y="Total Thumbs Up", ax=axes[1, 1], color="purple", edgecolor="black")
    axes[1, 1].set_title("Review Engagement Score per App")
    axes[1, 1].set_ylabel("Engagement Score")
    axes[1, 1].tick_params(axis="x", rotation=90,labelsize=6)

    # Line Chart: Install Growth Rate
    kpi_data.plot(kind="line", x="App Name", y="Growth Rate %", marker="o", ax=axes[2, 0], color="red", linestyle="dashed")
    axes[2, 0].set_title("App Install Growth Rate")
    axes[2, 0].set_ylabel("Growth Rate (%)")
    axes[2, 0].tick_params(axis="x", rotation=90,labelsize=6)

    # Bar Chart: User Retention Score
    kpi_data.plot(kind="bar", x="App Name", y="User Retention Score (%)", ax=axes[2, 1], color="teal", edgecolor="black")
    axes[2, 1].set_title("User Retention Score per App")
    axes[2, 1].set_ylabel("Retention Score")
    axes[2, 1].tick_params(axis="x", rotation=90,labelsize=6)

    # Adjust layout and display the plots
    # Adjust layout with better spacing
    plt.subplots_adjust(left=0.05, right=0.95, top=0.92, bottom=0.08, wspace=0.3, hspace=0.7)
    plt.show()

# Call the function with kpi_data
visualize_kpi_data(kpi_data)

### Predictive Analysis - Time Series Analysis to predict future user reviews

In [ ]:
import statsmodels.api as sm

# Prepare data for time series forecasting
time_series_data = df_merged.groupby(df_merged["at"].dt.to_period("M"))["reviewId"].count()
time_series_data.index = time_series_data.index.to_timestamp()  # Convert PeriodIndex to timestamp

# Fit an ARIMA model for forecasting
model = sm.tsa.ARIMA(time_series_data, order=(2, 1, 2))  # Auto-regressive integrated moving average
model_fit = model.fit()

# Forecast for next 6 months
forecast_steps = 6
forecast = model_fit.forecast(steps=forecast_steps)

# Plot the historical and forecasted review volume
plt.figure(figsize=(10, 5))
plt.plot(time_series_data, label="Historical Review Volume")
plt.plot(pd.date_range(start=time_series_data.index[-1], periods=forecast_steps+1, freq="ME")[1:], forecast, label="Forecasted Review Volume", linestyle="dashed")
plt.xlabel("Date")
plt.ylabel("Number of Reviews")
plt.title("Review Volume Forecast (Next 6 Months)")
plt.legend()
plt.show()